In [1]:
from aestheval.data.datasets import PCCD
import torch
SPLITS = ('train', 'validation', 'test')

pccd_dataset = [PCCD(split, data_path= "../data/PCCD",) for split in SPLITS]

In [2]:
dataset = torch.utils.data.ConcatDataset(pccd_dataset)

In [8]:
import statistics as st
from tqdm import tqdm 

def compute_mean(list_of_scores):
    good_scores = []
    for i in list_of_scores:
        try:
            good_scores.append(int(i))
        except:
            continue
    return st.mean(good_scores)

gt_scores, pred_scores = zip(*[(compute_mean(data['score']), data['mean_score']) for _, data in tqdm(dataset)])



100%|██████████| 4235/4235 [01:44<00:00, 40.37it/s]


In [9]:
from scipy import stats
import sklearn.metrics as sm

srcc = stats.spearmanr(gt_scores, pred_scores)[0]
print("SRCC =", srcc)
mse = round(sm.mean_squared_error(gt_scores, pred_scores), 4)
print("MSE =", mse)
lcc = stats.pearsonr(gt_scores, pred_scores)[0]
print("LCC =", lcc)

SRCC = 0.6066458147931981
MSE = 53.1099
LCC = 0.6499357318742437
